In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib notebook

import datareservoirio as drio

In [2]:
drio.globalsettings.environment.set_test()  # Work with test environment

# Create table
Lets create a table in Pandas -> DataFrame. 

In [3]:
table_dummy = pd.DataFrame(
    {
        "temp": 20.0 + np.random.randn(100),
        "strain": np.linspace(0.001, 0.01, num=100),
        "stress": np.linspace(10.0, 359.0, num=100),
    }
)

In [4]:
table_dummy.head(10)

,strain,stress,temp
0,0.001000,10.000000,20.555778
1,0.001091,13.525253,21.085800
2,0.001182,17.050505,20.291327
3,0.001273,20.575758,18.878574
4,0.001364,24.101010,21.370149
5,0.001455,27.626263,20.397454
6,0.001545,31.151515,20.818865
7,0.001636,34.676768,19.783192
8,0.001727,38.202020,19.190838
9,0.001818,41.727273,19.519266


# Simple table uploader

In [5]:
def table_uploader(auth, table, table_name):
    client = drio.Client(auth)
    
    table_summary = {'name': table_name,
                     'columns': {}}
    for col in table:
        print col
        response = client.create(pd.Series(table[col]))
        table_summary['columns'][col] = response['TimeSeriesId']
    return table_summary

# Upload to the reservoir

First, authenticate yourself towards the reservoir.

In [6]:
auth_key = drio.Authenticator("reservoir-integrationtest@4subsea.com")

Password: ········


Then initiate a client and use the authentication key.

In [7]:
table_summary = table_uploader(auth_key, table_dummy, "SuperDuperTable")

strain
stress
temp


Upload the DataFrame and store the response from the reservoir.

In [8]:
table_summary

{'columns': {'strain': u'd01d55c7-ca11-4913-9ec0-fbab5b09833d',
  'stress': u'890a72e5-97fd-4bf9-ae07-149195bfc434',
  'temp': u'69e69ecc-2843-417b-8473-bd54f56c6c7f'},
 'name': 'SuperDuperTable'}

The reservoir assigns a unique id - TimeSeriesId. This id can be used to make requests towards the reservoir.

# Retrieve data back from the reservoir
You can download data from the reservoir. Data is presented as Pandas Series object. The index is by default kept as integers.

## Simple table downloader

In [11]:
def table_download(auth, table_summary):
    client = drio.Client(auth)
    table = {}
    
    for col in table_summary['columns']:
        print col
        table[col] = client.get(table_summary['columns'][col])
    df = pd.DataFrame(table)
    df.index.name = None
    return df

In [12]:
table = table_download(auth_key, table_summary)

strain
stress
temp


In [13]:
table.head(10)

,strain,stress,temp
0,0.001000,10.000000,20.555778
1,0.001091,13.525253,21.085800
2,0.001182,17.050505,20.291327
3,0.001273,20.575758,18.878574
4,0.001364,24.101010,21.370149
5,0.001455,27.626263,20.397454
6,0.001545,31.151515,20.818865
7,0.001636,34.676768,19.783192
8,0.001727,38.202020,19.190838
9,0.001818,41.727273,19.519266


# Delete data in the reservoir

In [14]:
def table_delete(auth, table_summary):
    client = tsc.TimeSeriesClient(auth)

    for col in table_summary['columns']:
        print col
        table[col] = client.delete(table_summary['columns'][col])